# 第二章、基于问答助⼿的 Prompt 构建

在 `C4 数据库的搭建` 章节，我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings    # 调用 OpenAI 的 Embeddings 模型
import openai
from dotenv import load_dotenv, find_dotenv
import os

从环境变量中加载你的 OPENAI_API_KEY

In [2]:
_ = load_dotenv(find_dotenv())    # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

加载向量数据库，其中包含了 ../../data_base/knowledge_db 下多个文档的 Embedding

In [3]:
# 定义 Embeddings
embedding = OpenAIEmbeddings() 

# 向量数据库持久化路径
persist_directory = '../../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)

In [4]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：1121


我们可以测试一下加载的向量数据库，使用一个问题 query 进行向量检索。如下代码会在向量数据库中根据相似性进行检索，返回前 k 个最相似的文档。

> ⚠️使用相似性搜索前，请确保你已安装了 OpenAI 开源的快速分词工具 tiktoken 包：`pip install tiktoken`

In [6]:
question = "什么是强化学习"
docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：3


打印一下检索到的内容

In [7]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
 B站的小伙伴们好

我是蘑菇书一语二语二强化学习教程的作者之一王奇

今天来有给大家带来一个强化学习的入门指南

本次入门指南基于蘑菇书一语二语二强化学习教程

本书的作者目前都是Dell会员成员

也都是数学在读

下面去介绍每个作者

我是王奇

目前就留于中国科研院大学

引用方向是深度学习、静态视觉以及数据挖掘

杨玉云目前就读于清华大学

他的引用方向为

时空数据挖掘、智能冲砍系统以及
--------------
检索到的第1个内容: 
 而人工智能的基本挑战是

学习在不确定的情况下做出好的决策

这边我举个例子

比如你想让一个小孩学会走路

他就需要通过不断尝试来发现

怎么走比较好

怎么走比较快

强化学习的交互过程可以通过这张图来表示

强化学习由智能体和环境两部分组成

在强化学习过程中

智能体与环境一直在交互

智能体在环境中获取某个状态后

它会利用刚刚的状态输出一个动作

这个动作也被称为决策

然后这个动作会
--------------
检索到的第2个内容: 
 围棋游戏中比较出名的一个

强化学习的算法就是AlphaGo

此外我们可以使用强化学习

来控制机器人

以及来实现助力交通

另外还可以使用强化学习

来更好地给我们做推进

接下来就到第二部分

也就是为什么要使用本书来学习强化学习

这部分其实也是讲

这个蘑菇书它出版的一些故事

当时我在学习强化学习的时候

搜集了一些资料

然后我发现这些资料

都有点灰色难懂

并不是那么容易地上手
--------------


## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [8]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0 )
llm.predict("你好")

'你好！有什么我可以帮助你的吗？'

## 3. 构建 prompt

In [9]:
from langchain.prompts import PromptTemplate

# template = """基于以下已知信息，简洁和专业的来回答用户的问题。
#             如果无法从中得到答案，请说 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分。
#             答案请使用中文。
#             总是在回答的最后说“谢谢你的提问！”。
# 已知信息：{context}
# 问题: {question}"""
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)

# 运行 chain


再创建一个基于模板的检索链：

In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


创建检索 QA 链的方法 RetrievalQA.from_chain_type() 有如下参数：
- llm：指定使用的 LLM
- 指定 chain type : RetrievalQA.from_chain_type(chain_type="map_reduce")，也可以利用load_qa_chain()方法指定chain type。
- 自定义 prompt ：通过在RetrievalQA.from_chain_type()方法中，指定chain_type_kwargs参数，而该参数：chain_type_kwargs = {"prompt": PROMPT}
- 返回源文档：通过RetrievalQA.from_chain_type()方法中指定：return_source_documents=True参数；也可以使用RetrievalQAWithSourceChain()方法，返回源文档的引用（坐标或者叫主键、索引）

## 4. prompt 效果测试

In [11]:
question_1 = "什么是南瓜书？"
question_2 = "王阳明是谁？"

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [12]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
南瓜书是对《机器学习》（西瓜书）中难以理解的公式进行解析和补充推导细节的一本书。谢谢你的提问！


In [13]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
我不知道王阳明是谁，谢谢你的提问！


### 4.2 大模型自己回答的效果

In [14]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
llm.predict(prompt_template)

"南瓜书是指《深入理解计算机系统》（Computer Systems: A Programmer's Perspective）一书的俗称。这本书是由Randal E. Bryant和David R. O'Hallaron合著的计算机科学教材，旨在帮助读者深入理解计算机系统的工作原理和底层机制。南瓜书因其封面上有一个南瓜图案而得名，被广泛用于大学的计算机科学和工程课程中。"

In [15]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
llm.predict(prompt_template)

'王阳明（1472年-1529年），字仲明，号阳明子，是明代中期著名的思想家、政治家、军事家和教育家。他提出了“心即理”、“知行合一”的思想，强调人的内心自觉和道德修养的重要性。他的思想对中国历史产生了深远的影响，被后世尊称为“阳明先生”。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。